In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
uploaded = files.upload()

Saving telecom_combined.csv to telecom_combined (2).csv


In [ ]:
df = pd.read_csv("telecom_combined.csv")

In [ ]:
import pandas as pd
import numpy as np

# --- Step 1. Base Cleaning ---
df = df.sort_values(['service_provider', 'circle', 'date']).reset_index(drop=True)
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# --- Step 2. Temporal Lag Features (safe time-based info only) ---
for lag in [1, 3, 6, 12]:
    df[f'value_lag_{lag}'] = (
        df.groupby(['service_provider', 'circle'])['value']
        .shift(lag)
    )

# --- Step 3. Fill lags safely (optional bfill) ---
#lag_cols = [f'value_lag_{i}' for i in [1, 3, 6, 12]]
#df[lag_cols] = df.groupby(['service_provider', 'circle'])[lag_cols].bfill()

# --- Step 4. Date Components ---
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

# --- Step 5. Circle Type ---
metro_circles = ['Delhi', 'Mumbai', 'Chennai', 'Kolkata', 'Hyderabad', 'Bangalore']
df['circle_type'] = df['circle'].apply(
    lambda x: 'Metro' if x in metro_circles else 'Non-Metro'
)

# --- Step 6. Connection Type (safe binary encoding) ---
if 'type_of_connection' in df.columns:
    df['is_wireless'] = df['type_of_connection'].str.lower().eq('wireless').astype(int)
else:
    df['is_wireless'] = np.nan

# --- Step 7. Operator Geographic Diversity (# of circles) ---
operator_circle_count = df.groupby('service_provider')['circle'].nunique()
df['operator_geographic_diversity'] = df['service_provider'].map(operator_circle_count)

# --- Step 8. Market Size Category (circle-level total share proxy) ---
circle_totals = df.groupby('circle')['value'].sum().rank(pct=True)
df['market_size_category'] = df['circle'].map(circle_totals)

# --- Step 9. Competitive (non-target) features ---
df['market_share'] = df.groupby(['circle', 'date'])['value'].transform(lambda x: x / x.sum())
df['market_rank'] = df.groupby(['circle', 'date'])['market_share'].rank(ascending=False, method='dense')

leader_share = df.groupby(['circle', 'date'])['market_share'].transform('max')
df['share_gap_leader'] = leader_share - df['market_share']

circle_avg = df.groupby(['circle', 'date'])['value'].transform('mean')
df['relative_performance'] = df['value'] / circle_avg

# --- Step 10. Market Concentration (HHI) ---
hhi_df = (
    df.groupby(['circle', 'date'])['market_share']
    .apply(lambda x: np.sum(x**2))
    .reset_index(name='circle_hhi')
)
df = df.merge(hhi_df, on=['circle', 'date'], how='left')

df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.sort_values(['service_provider', 'circle', 'date']).reset_index(drop=True)

# --- Step 2 : Compute month-over-month subscriber % change ---
df['subscriber_change'] = (
    df.groupby(['service_provider', 'circle'])['value']
    .pct_change()
    .fillna(0)
)

# --- Step 3 : Define churn target ---
# Churn = 1 if subscriber count drops > 15 % month-over-month
df['churn'] = (df['subscriber_change'] < -0.15).astype(int)

print("✅ Stage 1 Safe Features Created Successfully!")
print("Shape:", df.shape)



✅ Stage 1 Safe Features Created Successfully!
Shape: (42122, 22)


In [ ]:

# --- Save this leakage-free version ---
df_safe = df.copy()
df_safe.to_csv("telecom_safe_features.csv", index=False)
print("💾 Saved telecom_safe_features.csv for modeling use.")

💾 Saved telecom_safe_features.csv for modeling use.


In [ ]:
from google.colab import files

# Download it
files.download('telecom_safe_features.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import numpy as np

# --- Step 1. Load the Stage 1 Data ---
df = pd.read_csv("telecom_features_stage1.csv")

# Ensure date is datetime
df['date'] = pd.to_datetime(df['date'])

# Sort to ensure chronological order
df = df.sort_values(['service_provider', 'circle', 'date']).reset_index(drop=True)

# --- Step 2. Calculate Month-over-Month Subscriber Change ---
df['subscriber_change'] = (
    df.groupby(['service_provider', 'circle'])['value']
    .pct_change()
    .fillna(0)
)

# --- Step 3. Define Churn Target ---
# Churn = 1 if subscriber count drops > 15% month-over-month
df['churn'] = (df['subscriber_change'] < -0.15).astype(int)

# --- Step 4. Define Churn Severity (categorical interpretation) ---
def categorize_churn(change):
    if change <= -0.30:
        return 'High'
    elif change <= -0.15:
        return 'Medium'
    elif change <= -0.05:
        return 'Low'
    elif change < 0:
        return 'Mild'
    else:
        return 'Stable'

df['churn_severity'] = df['subscriber_change'].apply(categorize_churn)

# --- Step 5. Sustained Decline (2+ months continuous drop) ---
df['declining_trend'] = (
    df.groupby(['service_provider', 'circle'])['subscriber_change']
    .rolling(window=2)
    .apply(lambda x: (x < 0).all())
    .reset_index(level=[0,1], drop=True)
    .astype(bool)
)

# --- Step 6. Optional: Operator Category (Major vs Regional) ---
major_ops = ['JIO', 'AIRTEL', 'VI', 'BSNL']
df['operator_category'] = df['service_provider'].apply(
    lambda x: 'Major' if any(op in x.upper() for op in major_ops) else 'Regional'
)

# --- Step 7. Final Checks ---
print("✅ Stage 2 — Churn Targets Added Successfully!")
print(df[['service_provider', 'circle', 'date', 'value', 'subscriber_change', 'churn', 'churn_severity']].head())

# --- Step 8. Save Final Modeling Dataset ---
df.to_csv("telecom_features_stage2.csv", index=False)
print("💾 Saved final dataset as telecom_features_stage2.csv")


## BREAK

In [ ]:
df = df.sort_values(['service_provider', 'circle', 'date']).reset_index(drop=True)

In [ ]:
# Check how many missing values exist in total
print("Total missing values:", df.isna().sum().sum())

# View missing count per column (sorted)
missing_summary = df.isna().sum().sort_values(ascending=False)
print(missing_summary.head(15))

Total missing values: 0
year                  0
month                 0
circle                0
type_of_connection    0
service_provider      0
value                 0
date                  0
dtype: int64


In [ ]:
for lag in [1, 3, 6, 12]:
    df[f'value_lag_{lag}'] = (
        df.groupby(['service_provider', 'circle'])['value'].shift(lag)
    )

In [ ]:
df['subscriber_change'] = (
    (df['value'] - df['value_lag_1']) / df['value_lag_1']
)
df['subscriber_change'] = df['subscriber_change'].fillna(0)

In [ ]:
df['churn'] = (df['subscriber_change'] < -0.15).astype(int)

In [ ]:
df['churn'].isna().sum()

np.int64(0)

In [ ]:
def categorize_churn_severity(change):
    if change <= -0.30:
        return 'High'
    elif change <= -0.15:
        return 'Medium'
    elif change <= -0.05:
        return 'Low'
    elif change < 0:
        return 'Mild'
    else:
        return 'Stable'

df['churn_severity'] = df['subscriber_change'].apply(categorize_churn_severity)

# Sustained decline for 2+ months
df['declining_trend'] = (
    df.groupby(['service_provider', 'circle'])['subscriber_change']
      .rolling(window=2)
      .apply(lambda x: (x < 0).all())
      .reset_index(level=[0,1], drop=True)
      .astype(bool)
)

In [ ]:
def validate_churn_patterns(df):
    major_ops = ['JIO', 'AIRTEL', 'VI', 'BSNL']
    df['operator_category'] = df['service_provider'].apply(
        lambda x: 'Major' if any(op in x.upper() for op in major_ops) else 'Regional'
    )

    churn_by_cat = df.groupby('operator_category')['churn'].mean()
    print("Churn rates by operator category:\n", churn_by_cat)

    churn_by_circle = df.groupby('circle')['churn'].mean().sort_values(ascending=False)
    print("Top circles by churn:\n", churn_by_circle.head(10))

    return df

df = validate_churn_patterns(df)

Churn rates by operator category:
 operator_category
Major       0.397355
Regional    0.269471
Name: churn, dtype: float64
Top circles by churn:
 circle
Jharkhand               0.444444
Madhya Pradesh          0.367764
Punjab                  0.367534
Uttar Pradesh (West)    0.362493
Maharashtra             0.360197
Kerala                  0.359743
Gujarat                 0.359203
Uttar Pradesh (East)    0.354892
Karnataka               0.353806
Rajasthan               0.352296
Name: churn, dtype: float64


In [ ]:
for lag in [3, 6, 12]:
    df[f'growth_volatility_{lag}'] = (
        df.groupby(['service_provider', 'circle'])['subscriber_change']
        .transform(lambda x: x.rolling(window=lag, min_periods=2).std())
    )

# Month-over-month and year-over-year growth
df['mom_growth'] = df.groupby(['service_provider', 'circle'])['value'].pct_change(1)
df['yoy_growth'] = df.groupby(['service_provider', 'circle'])['value'].pct_change(12)

# 12-month trend (slope)
def calc_trend(series):
    y = series.values
    if np.isnan(y).sum() > len(y) - 3:
        return np.nan
    x = np.arange(len(y))
    valid = ~np.isnan(y)
    if valid.sum() < 3:
        return np.nan
    return np.polyfit(x[valid], y[valid], 1)[0]

df['trend_12m'] = (
    df.groupby(['service_provider', 'circle'])['value']
    .transform(lambda x: x.rolling(window=12, min_periods=3).apply(calc_trend, raw=False))
)

In [ ]:
df['growth_momentum'] = df.groupby(['service_provider', 'circle'])['subscriber_change'].diff()
df['growth_acceleration'] = df.groupby(['service_provider', 'circle'])['growth_momentum'].diff()

# Seasonal adjustment
monthly_mean = df.groupby('month')['value'].mean()
df['seasonal_factor'] = df['month'].map(monthly_mean)
df['seasonally_adjusted_value'] = df['value'] / df['seasonal_factor']

In [ ]:
def create_competitive_features(df):
    df_comp = df.copy()

    # ✅ Ensure data is time-sorted to avoid leakage in any rolling/group ops
    df_comp = df_comp.sort_values(['circle', 'date']).reset_index(drop=True)

    # ----- 1️⃣ Market Share -----
    # Share of each operator within a circle on a given date
    df_comp['market_share'] = df_comp.groupby(['circle', 'date'])['value'].transform(
        lambda x: x / x.sum()
    )

    # ----- 2️⃣ Market Rank (1 = leader) -----
    df_comp['market_rank'] = df_comp.groupby(['circle', 'date'])['market_share'].rank(
        ascending=False, method='dense'
    )

    # ----- 3️⃣ Share Gap from Market Leader -----
    leader_share = df_comp.groupby(['circle', 'date'])['market_share'].transform('max')
    df_comp['share_gap_leader'] = leader_share - df_comp['market_share']

    # ----- 4️⃣ Relative Performance vs Circle Average -----
    circle_avg = df_comp.groupby(['circle', 'date'])['value'].transform('mean')
    df_comp['relative_performance'] = df_comp['value'] / circle_avg

    # ----- 5️⃣ Market Concentration (HHI proxy) -----
    # Herfindahl–Hirschman Index = Σ (market_share²) per circle-date
    hhi_df = (
        df_comp.groupby(['circle', 'date'])['market_share']
        .apply(lambda x: np.sum(x**2))
        .reset_index(name='circle_hhi')
    )

    df_comp = df_comp.merge(hhi_df, on=['circle', 'date'], how='left')

    print("✅ Competitive features created successfully.")
    return df_comp


In [ ]:
df_competitive_features = create_competitive_features(df)
print("New shape after competitive features:", df_competitive_features.shape)


✅ Competitive features created successfully.
New shape after competitive features: (42122, 31)


In [ ]:
df_competitive_features['market_rank'] = df_competitive_features.groupby(['circle', 'date'])['market_share'].rank(ascending=False)
leader_share = df_competitive_features.groupby(['circle', 'date'])['market_share'].transform('max')
df_competitive_features['share_gap_leader'] = leader_share - df_competitive_features['market_share']

circle_avg = df_competitive_features.groupby(['circle', 'date'])['value'].transform('mean')
df_competitive_features['relative_performance'] = df_competitive_features['value'] / circle_avg

In [ ]:
def create_business_features(df):
    df_biz = df.copy()

    # ----- 1️⃣ Circle Classification (Metro vs Non-Metro) -----
    metro_circles = ['Delhi', 'Mumbai', 'Chennai', 'Kolkata', 'Hyderabad', 'Bangalore']
    df_biz['circle_type'] = df_biz['circle'].apply(
        lambda x: 'Metro' if x in metro_circles else 'Non-Metro'
    )

    # ----- 2️⃣ Connection Type (Wireless vs Wireline) -----
    if 'type_of_connection' in df_biz.columns:
        df_biz['is_wireless'] = df_biz['type_of_connection'].str.lower().eq('wireless').astype(int)
    else:
        df_biz['is_wireless'] = np.nan

    # ----- 3️⃣ Market Size Category (Total Circle Subscribers) -----
    # Compute using current dataset only (safe pre-split)
    total_market_size = (
        df_biz.groupby('circle')['value']
        .sum()
        .rank(pct=True)
    )
    df_biz['market_size_category'] = df_biz['circle'].map(total_market_size)

    # ----- 4️⃣ Operator Geographic Diversity -----
    operator_circle_count = df_biz.groupby('service_provider')['circle'].nunique()
    df_biz['operator_geographic_diversity'] = df_biz['service_provider'].map(operator_circle_count)

    print("✅ Business & contextual features created successfully.")
    return df_biz


# Apply to your competitive features dataset
df__features = create_business_features(df_competitive_features)
print("Final shape after all feature creation:", df__features.shape)
df__features.head()

✅ Business & contextual features created successfully.
Final shape after all feature creation: (42122, 35)


,year,month,circle,type_of_connection,service_provider,value,date,value_lag_1,value_lag_3,value_lag_6,...,seasonally_adjusted_value,market_share,market_rank,share_gap_leader,relative_performance,circle_hhi,circle_type,is_wireless,market_size_category,operator_geographic_diversity
0,2009,January,All India,wireless,Aircel,16761397.0,1/1/2009,NaN,NaN,NaN,...,1.965483,0.041899,8.0,0.179033,0.796077,0.137829,Non-Metro,1,1.0,26
1,2009,January,All India,wireless,BPL Mobile,2007303.0,1/1/2009,NaN,NaN,NaN,...,0.235381,0.005018,13.0,0.215914,0.095336,0.137829,Non-Metro,1,1.0,2
2,2009,January,All India,wireless,BSNL,47584490.0,1/1/2009,NaN,NaN,NaN,...,5.579876,0.118948,4.0,0.101984,2.260008,0.137829,Non-Metro,1,1.0,35
3,2009,January,All India,wireline,BSNL,29291013.0,1/1/2009,47584490.0,NaN,NaN,...,3.434737,0.073219,7.0,0.147713,1.391166,0.137829,Non-Metro,0,1.0,35
4,2009,January,All India,wireless,Bharti Airtel,88382758.0,1/1/2009,NaN,NaN,NaN,...,10.363983,0.220932,1.0,0.000000,4.197708,0.137829,Non-Metro,1,1.0,28


In [ ]:
print("Total missing values:", df__features.isna().sum().sum())
print(df__features.isna().sum().sort_values(ascending=False).head(10))

Total missing values: 21479
value_lag_12            5626
yoy_growth              5626
value_lag_6             3015
value_lag_3             1614
trend_12m               1116
growth_acceleration     1116
growth_volatility_6      561
mom_growth               561
growth_volatility_3      561
growth_volatility_12     561
dtype: int64


In [ ]:
df__features = df__features.fillna(0)


In [ ]:
print("Total missing values after fill:", df__features.isna().sum().sum())


Total missing values after fill: 0


In [ ]:
# ✅ Save final processed dataframe to CSV
df__features.to_csv("telecom_features.csv", index=False)

# If you're using Colab / Jupyter and want to download it directly:
from google.colab import files
files.download("telecom_features.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>